In [ ]:
# 🚀 SpaceX Launch Dashboard (Google Colab Version)# This notebook runs an interactive Plotly Dash app inline in Google Colab using JupyterDash.# It visualizes SpaceX Falcon 9 launch outcomes by site and payload mass.!pip install jupyter-dash dash pandas plotlyimport pandas as pdfrom jupyter_dash import JupyterDashfrom dash import html, dcc, Input, Outputimport plotly.express as px# Load SpaceX dataseturl = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"spacex_df = pd.read_csv(url)max_payload = spacex_df['Payload Mass (kg)'].max()min_payload = spacex_df['Payload Mass (kg)'].min()# Initialize appapp = JupyterDash(__name__)app.layout = html.Div(children=[    html.H1('SpaceX Launch Records Dashboard',            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),    dcc.Dropdown(        id='site-dropdown',        options=[{'label': 'All Sites', 'value': 'ALL'}] +                [{'label': s, 'value': s} for s in spacex_df['Launch Site'].unique()],        value='ALL', searchable=True    ),    html.Br(),    html.Div(dcc.Graph(id='success-pie-chart')),    html.Br(),    html.P('Payload range (Kg):'),    dcc.RangeSlider(        id='payload-slider', min=0, max=10000, step=1000,        marks={i: str(i) for i in range(0, 10001, 2500)},        value=[min_payload, max_payload]    ),    html.Br(),    html.Div(dcc.Graph(id='success-payload-scatter-chart'))])# Callback for pie chart@app.callback(Output('success-pie-chart', 'figure'),              Input('site-dropdown', 'value'))def update_pie_chart(selected_site):    if selected_site == 'ALL':        fig = px.pie(spacex_df, names='Launch Site', values='class',                     title='Total Success Launches by Site')    else:        filtered = spacex_df[spacex_df['Launch Site'] == selected_site]        fig = px.pie(filtered, names='class',                     title=f'Success vs Failure for {selected_site}')    return fig# Callback for scatter plot@app.callback(Output('success-payload-scatter-chart', 'figure'),              [Input('site-dropdown', 'value'), Input('payload-slider', 'value')])def update_scatter_chart(selected_site, payload_range):    low, high = payload_range    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]    if selected_site == 'ALL':        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',                         color='Booster Version Category',                         title='Payload vs. Success for All Sites')    else:        site_df = filtered_df[filtered_df['Launch Site'] == selected_site]        fig = px.scatter(site_df, x='Payload Mass (kg)', y='class',                         color='Booster Version Category',                         title=f'Payload vs. Success for {selected_site}')    return fig# Run inlineapp.run_server(mode='inline')